In [5]:
import scipy.io
import matplotlib.pyplot as plt
import numpy as np

In [6]:
mat = scipy.io.loadmat('mnist-original.mat') # lecture du fichier

FileNotFoundError: [Errno 2] No such file or directory: 'mnist-original.mat'

In [ ]:
data = np.transpose(mat['data'])
label = np.transpose(mat['label'])

In [ ]:
X = [] # on créé une nouvelle liste qui va contenir nos matrices 28x28
for i in range(len(data)):
    X.append(data[i].reshape(28,28))

In [ ]:
def index_chiffre(n, y):  # elle nous renvoie un tableau contenant les indices représentants le chiffre n
    index = []           # j'ai rajouté le y dans les arguments car je me servirai de cette fonction dans la suite
    for i in range(len(y)):
        if y[i]==n:
            index.append(i)
    return index

In [ ]:
def image_moyenne(n, X, y): # cette fonction renvoie la matrice moyenne (centroïde) pour le chiffre n
    index = index_chiffre(n, y) # de même je rajoute l'argument X et y car je m'en servirai dans la suite
    sum_matrice = np.zeros((28,28)) # la matrice qui sera utilisée dans la somme
    for i in index:
        sum_matrice += X[i] # on somme toutes les matrices représentants le chiffre n
    return sum_matrice/len(index)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = [], [], [], [] # comme la base de donnée est ordonnée, pour pouvoir diviser en train et test
for i in range(10): # on doit diviser en train test chaque set de chiffre, pour 0 pour 1 etc et on met tout dans nos matrices défeinies juste avant
    temp_x = [] # variable temporaire qui va contenir les matrices représentants le chiffre i
    temp_y = [] # variable temporaire qui va contenir les indexes représentants le chiffre i
    for j in (index_chiffre(i,label)):
        temp_x.append(X[j])
        temp_y.append(label[j])
    x_train, x_test, Y_train, Y_test = train_test_split(temp_x, temp_y, test_size=0.2) # on divise en train test pour chaque jeu de matrice représentant le chiffre i
    X_train += x_train # à chaque itération on les mets dans nos matrices précédemment définies
    X_test += x_test
    y_train += Y_train
    y_test += Y_test

In [ ]:
centroide_train = []
for i in range(10): # on calcul le centroïde pour chaque chiffre i entre 0 et 9 avec X_train et y_train
    centroide_train.append(image_moyenne(i, X_train, y_train)) 

In [ ]:
fig, ax = plt.subplots(1,10,figsize=(20,2)) # affichage des centroïdes 
for i in range(10):
    ax[i].imshow(centroide_train[i],cmap='gray')
    ax[i].set_title("mean "+str(i))

In [ ]:
def distance(x,y,k): # fonction qui calcule la distance entre deux matrices 
    x = x.reshape(len(x)**2) # x est une matrice carré de "longueur" n donc sa représentation en vecteur sera de taille n*n
    y = y.reshape(len(y)**2)
    return np.linalg.norm(x-y,k)

##### Dans les deux prochaines lignes, on calcule le pourcentage de réussite en utilisant la norme euclidienne entre deux vecteur, pour ça on stack les lignes des matrices l'une sur l'autre pour obtenir un vecteur 784, puis on utilise la norme euclidienne entre les deux vecteurs obtenus, pour une raison que j'ignore cette méthode prend beaucoup de temps (environ 7mn pour calculer le pourcentage de réussite).

In [ ]:
def prediction(x, n, k): # on va chercher le centroïde le plus proche de la matrice X_n où n est l'indice de la matrice
    minimum = distance(x[n], centroide_train[0], k) # on va chercher le minimum des distances entre X_n et chaque matrice du centroïde
    j = 0 # l'indice qui nous permettra de savoir quel case de centroide_train notre X_n représente
    for i in range(10):
        if distance(x[n], centroide_train[i], k) < minimum:
            minimum = distance(x[n], centroide_train[i], k)
            j=i # lorsqu'on a trouvé ce minimum on enregistre à quelle case il est atteint
    return float(j) # on le retourne en float pour pouvoir le comparer plus tard aux valeurs de y_test qui sont en float

In [ ]:
# On va calculer le pourcentage de réussite 
j = 0 # elle va compte le nombre de fois où l'aglorithme a bien prédit
for i in range(len(X_test)):
    if y_test[i]==prediction(X_test, i, 2):
        j+=1
print("Le pourcentage de précision est :"+str(j*100/len(X_test)))

##### Ici on utilise une méthode différente pour calcule rla distance entre deux matrices, d'abord je définis la norme matricielle (celle de Frobenius), ensuite je définis la distance entre deux matrices comme étant la norme de la différence, après ça j'utilise le même code que précédemment sauf que je remplace l'ancienne distance par la nouvelle, au final cette méthode est extrêment plus rapide que la précédente (environ 4s).

In [ ]:
def norme_eucli_matrice(A): # calcul de la norme euclidienne de la matrice
    return np.trace(A@(A.transpose()))**(1/2)

In [ ]:
def distance_norme(x,y):
    return norme_eucli_matrice(x-y)

In [ ]:
def prediction_norme(x, n): # on va chercher le centroïde le plus proche de la matrice X_n où n est l'indice de la matrice
    minimum = distance_norme(x[n], centroide_train[0]) # on va chercher le minimum des distances entre X_n et chaque matrice du centroïde
    j = 0 # l'indice qui nous permettra de savoir quel case de centroide_train notre X_n représente
    for i in range(10):
        if distance_norme(x[n], centroide_train[i]) < minimum:
            minimum = distance_norme(x[n], centroide_train[i])
            j=i # lorsqu'on a trouvé ce minimum on enregistre à quelle case il est atteint
    return float(j) # on le retourne en float pour pouvoir le comparer plus tard aux valeurs de y_test qui sont en float

In [ ]:
# On va calculer le pourcentage de réussite en utilisant la norme matricielle
k = 0 # elle va compte le nombre de fois où l'aglorithme a bien prédit
for i in range(len(X_test)):
    if y_test[i]==prediction_norme(X_test, i):
        k+=1
print("Le pourcentage de précision est "+'{:.2f}'.format(k*100/len(X_test)))

##### On va maintenant utiliser plusieur norme L^p

In [ ]:
def calcul_pourcentage(n): # n représente l'ordre de la norme qu'on va choisir
    k = 0
    for i in range(len(X_test)):
        if y_test[i]==prediction(X_test, i, n):
            k+=1
    return float('{:.2f}'.format(k*100/len(X_test)))

In [ ]:
pourcentage = []
plt.figure(figsize=(12,6))
for i in range(1,15):
    pourcentage.append(calcul_pourcentage(i))  
plt.plot(pourcentage)


In [ ]:
# on voit que le maximum est atteint pour la troisième valeur donc on a de meilleurs résultats pour une norme L^3

In [ ]:
for i in range(1,7):
    print("Le pourcentage de précision pour la norme L^"+str(i)+" est égal à "+str(pourcentage[i-1]))





**La matrice de confusion**:


Elle permet de jauger directement les nombres les plus à même d'être confondus : si la prédiction était sans failles, l'on obtiendrait des nombres le long de la diagonale et des 0 partout ailleurs. 

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test,[prediction_norme(X_test,i) for i in range(len(y_test))])

In [ ]:
import itertools

plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('matrice de confusion')
plt.colorbar()
    
fmt = 'd'
thresh = cm.max()/2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j,i,format(cm[i,j], fmt), horizontalalignment="center",color="grey" if cm[i,j]> thresh else "black")
        
plt.tight_layout()
plt.ylabel('Vrai nombre')
plt.xlabel('Nombre prédit')
plt.show()

##### On va utiliser un réseau de neuronnes

In [ ]:
X_train = np.array(X_train) # on transforme nos données en données numpy
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
X_train = X_train / 255 # on "normalise" nos données pour avoir des valeurs comprises entre 0 et 1
X_test = X_test / 255

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Softmax
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [ ]:
model = Sequential([
                    Flatten(input_shape=(28,28)),
                    Dense(728, activation='relu'),
                    Dense(10)
                    ])

In [ ]:
model.compile(optimizer='adam', loss=SparseCategoricalCrossentropy(from_logits=True), metrics='accuracy')

In [ ]:
model.fit(X_train, y_train, epochs=10)

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred[7643] # y_pred sort un tableau de 10 nombres qui représente la confiance du modèle que l'image correspond à chacun des 10 chiffres

In [ ]:
model_probability = Sequential([model, Softmax()])

In [ ]:
y_pred_proba = model_probability.predict(X_test)

In [ ]:
y_pred_proba[12050]

In [ ]:
def afficher_prediction(i):
    plt.imshow(X_test[i])
    plt.title("Le numéro affiché est "+str(np.argmax(y_pred_proba[i]))+", avec un pourcentage de "+str('{:.2f}'.format(y_pred_proba[i][np.argmax(y_pred_proba[i])]*100)))
    plt.show()

In [ ]:
afficher_prediction(12050)

In [ ]:
X_test[5]